In [1]:
import theano
import theano.tensor as T

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5103)


## Recall:
* SharedVariable resides on GPU
* data transfer between GPU <-> CPU takes much time

In [2]:
import numpy as np
def floatX(x):
    return np.asarray(x, dtype='float32')

w_val = floatX(np.ones((256,256)))
w = theano.shared(w_val, name='w')

---
## Observe that usually the data goes from CPU -> GPU

In [11]:
u = T.matrix('u')
y = 3*u + 4
compute_y = theano.function([u], y)
print compute_y(w_val)

[[ 7.  7.  7. ...,  7.  7.  7.]
 [ 7.  7.  7. ...,  7.  7.  7.]
 [ 7.  7.  7. ...,  7.  7.  7.]
 ..., 
 [ 7.  7.  7. ...,  7.  7.  7.]
 [ 7.  7.  7. ...,  7.  7.  7.]
 [ 7.  7.  7. ...,  7.  7.  7.]]


#### Can we pre-load the data on the GPU and feed it as an input whenever we want?
---
## `givens` option of `theano.function`

In [12]:
compute_y2 = theano.function([], y, givens={u: w})
print compute_y2()

[[ 19.  19.  19. ...,  19.  19.  19.]
 [ 19.  19.  19. ...,  19.  19.  19.]
 [ 19.  19.  19. ...,  19.  19.  19.]
 ..., 
 [ 19.  19.  19. ...,  19.  19.  19.]
 [ 19.  19.  19. ...,  19.  19.  19.]
 [ 19.  19.  19. ...,  19.  19.  19.]]


---
## How can we update the SharedVariable?
#### Naive approach

In [6]:
print "initially: \n", w.get_value()
w.set_value( w.get_value() + 1 )
print "now: \n", w.get_value()

initially: 
[[ 3.  3.  3. ...,  3.  3.  3.]
 [ 3.  3.  3. ...,  3.  3.  3.]
 [ 3.  3.  3. ...,  3.  3.  3.]
 ..., 
 [ 3.  3.  3. ...,  3.  3.  3.]
 [ 3.  3.  3. ...,  3.  3.  3.]
 [ 3.  3.  3. ...,  3.  3.  3.]]
now: 
[[ 4.  4.  4. ...,  4.  4.  4.]
 [ 4.  4.  4. ...,  4.  4.  4.]
 [ 4.  4.  4. ...,  4.  4.  4.]
 ..., 
 [ 4.  4.  4. ...,  4.  4.  4.]
 [ 4.  4.  4. ...,  4.  4.  4.]
 [ 4.  4.  4. ...,  4.  4.  4.]]


#### However, note that

* `get_value`: **GPU -> CPU**
* `set_value`: **CPU -> GPU**

So `w.set_value( w.get_value() + 1 )` is not a efficient way.

#### Can we update the value without getting it to the CPU?
---
## `update` option of `theano.function`

In [8]:
update_w = theano.function([], [], updates={w: w+1})

print "initially: \n", w.get_value()
update_w()
print "now: \n", w.get_value()

initially: 
[[ 4.  4.  4. ...,  4.  4.  4.]
 [ 4.  4.  4. ...,  4.  4.  4.]
 [ 4.  4.  4. ...,  4.  4.  4.]
 ..., 
 [ 4.  4.  4. ...,  4.  4.  4.]
 [ 4.  4.  4. ...,  4.  4.  4.]
 [ 4.  4.  4. ...,  4.  4.  4.]]
now: 
[[ 5.  5.  5. ...,  5.  5.  5.]
 [ 5.  5.  5. ...,  5.  5.  5.]
 [ 5.  5.  5. ...,  5.  5.  5.]
 ..., 
 [ 5.  5.  5. ...,  5.  5.  5.]
 [ 5.  5.  5. ...,  5.  5.  5.]
 [ 5.  5.  5. ...,  5.  5.  5.]]
